In [1]:
import pandas as pd
import numpy as np
from rapidfuzz import process, fuzz
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy.stats import ttest_rel

df_numbers = pd.read_csv("cleaned_numbers.csv")
df_numbers.head()

ModuleNotFoundError: No module named 'rapidfuzz'

In [ ]:
df_imtm = pd.read_csv("combinedimdbtmdb.csv")
df_imtm.head()

In [ ]:
# Quick exact title join using the titles for initial corelation checks
merged_check = pd.merge(df_numbers, df_imtm, left_on="movie", right_on="primary_title", how="inner")
merged_check.head()


In [ ]:
merged_check.info()

In [ ]:
#Use fuzzy match to compare titles

# Create a dictionary mapping  combined data title -> numbers_title()
title_map = {}

tmdb_titles = tmdb_df['title'].tolist()

for imdb_title in imdb_df['primary_title']:
    match, score, idx = process.extractOne(
        imdb_title, tmdb_titles, scorer=fuzz.token_sort_ratio
    )
    if score >= 85:  # threshold (0-100), adjust if too strict/loose
        title_map[imdb_title] = match


imdb_df['tmdb_match'] = imdb_df['primary_title'].map(title_map)

# Merge on imdb.primary_title -> tmdb.title
combined_df = imdb_df.merge(
    tmdb_df,
    left_on='tmdb_match',
    right_on='title',
    how='inner',
    suffixes=('_imdb','_tmdb')
)

In [ ]:
#Use fuzzy match to compare titles

# Create a dictionary mapping imdb_title -> best tmdb_title
title_map = {}

numbers_titles = df_numbers['movie'].tolist() #List of all movie titles from numbers for cross reference

for imtm_title in df_imtm['primary_title']:
    match, score, idx = process.extractOne(
        imtm_title, numbers_titles, scorer=fuzz.token_sort_ratio
    )
    if score >= 85:  # threshold (0-100), adjust if too strict/loose
        title_map[imtm_title] = match


df_imtm['movie_name_match'] = df_imtm['primary_title'].map(title_map)

# Merge on imdb.primary_title -> tmdb.title
combined_df = df_imtm.merge(
    df_numbers,
    left_on='movie_name_match',
    right_on='movie',
    how='inner',
    suffixes=('_numbers','_imdb/tmdb')
)

In [ ]:
combined_df.info()

In [ ]:
#combined_df.to_csv("finalcombined.csv", index=False)